In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import os

spark = SparkSession.builder \
    .appName("StackOverflow Survey Analysis") \
    .getOrCreate()

def load_csvs():
    dfs = {}
    
    for i in range(20, 25):
        file_path = f'../bronze/stack_overflow_data/survey_results_public{i}.csv'
        try:
            df = spark.read.option("header", "true") \
                         .option("inferSchema", "true") \
                         .csv(file_path)
            
            dfs[f'df_{i}'] = df
            
            print(f'Successfully loaded data for year 20{i}')
            print(f'Number of rows and columns in df_{i}: {(df.count(), len(df.columns))}')
            print(f'Preview of df_{i}:')
            df.show(5, truncate=False)
            print('\n' + '-'*50 + '\n')
            
        except Exception as e:
            print(f'Warning: Error loading file {file_path}: {str(e)}')
    
    return dfs


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\dell\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
load_csvs()

{}